In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses

2023-03-05 16:34:25.468461: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import cv2
import os
import numpy as np

In [3]:


def loadImages(path, urls,target ):
  images = []
  labels = []
  #for i in range(len(urls))
  for i in range(len(urls)):
    img_path = path + "/" + urls[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    # if we want to resize the images
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(target)
  images = np.asarray(images)
  return images, labels


covid_path = "./../data/COVID-19_Radiography_Dataset/COVID/images"
covidUrl = os.listdir(covid_path)
covidImages, covidTargets = loadImages(covid_path, covidUrl, 1)

normal_path = "./../data/COVID-19_Radiography_Dataset/Normal/images"
normal_urls = os.listdir(normal_path)
normalImages, normalTargets = loadImages(normal_path, normal_urls, 0)

covidImages=np.asarray(covidImages)

normalImages=np.asarray(normalImages)

data = np.r_[covidImages, normalImages]

targets = np.r_[covidTargets, normalTargets]


from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.25)


In [4]:
model = models.Sequential()

model.add(layers.experimental.preprocessing.Resizing(100,100, interpolation="bilinear", input_shape=x_train.shape[1:]))

model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))

model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))

model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 25, 25, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 25, 25, 96)        0         
                                                                 
 activation (Activation)     (None, 25, 25, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 256)         614656    
                                                        

2023-03-05 16:35:25.328262: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 16:35:25.464110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14786 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0


In [5]:
# Compile the model
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
                

In [6]:
model.fit(x_train, y_train, epochs=10,
            validation_data=(x_test, y_test))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-03-05 16:35:31.228867: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-03-05 16:35:31.515255: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0xc5386b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-05 16:35:31.515294: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-03-05 16:35:31.520997: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-05 16:35:31.664759: I tensorflow/compiler/jit/xla_compi

324/324 [==============================] - 8s 14ms/step - loss: 0.6108 - accuracy: 0.7292 - val_loss: 0.5666 - val_accuracy: 0.7491
Epoch 2/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5859 - accuracy: 0.7345 - val_loss: 0.5669 - val_accuracy: 0.7491
Epoch 3/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5847 - accuracy: 0.7345 - val_loss: 0.5727 - val_accuracy: 0.7491
Epoch 4/10
324/324 [==============================] - 3s 11ms/step - loss: 0.5816 - accuracy: 0.7345 - val_loss: 0.5676 - val_accuracy: 0.7491
Epoch 5/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5819 - accuracy: 0.7345 - val_loss: 0.5668 - val_accuracy: 0.7491
Epoch 6/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5821 - accuracy: 0.7345 - val_loss: 0.5718 - val_accuracy: 0.7491
Epoch 7/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5821 - accuracy: 0.7345 - val_loss: 0.5645 - val_accuracy: 0.7491
Epoch 8/10

In [7]:
# evaluate the model
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.734, Test: 0.749
